中国教育在线，对应的网址是：https://gkcx.eol.cn/school/search  

因为对应的是位置点，所以最主要的应该是经纬度点坐标数据。  

但是在该网站中无法获取到高校的经纬度数据，所以利用地址解析的方式获取到经纬度坐标。  

# 数据抓取

基于上面的爬虫流程，依次进行爬虫的代码实现  

首先是获取到总记录数

In [4]:
import json
import random
import time
import pandas as pd
import numpy as np
import warnings
import requests

爬虫头的封装

In [5]:
def get_ua():
    """
    在UA库中随机选择一个UA
    :return: 返回一个库中的随机UA
    """
    ua_list = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60",
        "Opera/8.0 (Windows NT 5.1; U; en)",
        "Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50",
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0",
        "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
        "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.133 Safari/534.16",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.101 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0)",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Maxthon/4.4.3.4000 Chrome/30.0.1599.101 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.122 UBrowser/4.0.3214.0 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36",
        "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",
        "Mozilla/5.0 (Windows; U; Windows NT 5.2) AppleWebKit/525.13 (KHTML, like Gecko) Chrome/0.2.149.27 Safari/525.13",
        "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
        "Mozilla/5.0 (Macintosh; U; IntelMac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1Safari/534.50",
        "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:51.0) Gecko/20100101 Firefox/51.0",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0",
        "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"]

    return random.choice(ua_list)


In [6]:
def get_size(page=1):
    """
    获取数据总条数
    @param page:
    @return:
    """
    url = "https://api.zjzw.cn/web/api/?keyword=&page={0}&size=20&uri=apidata/api/gkv3/school/lists".format(page)
    res = requests.post(url, headers={'User-Agent': get_ua()})
    data = json.loads(res.text)
    print(data)
    size = 0
    if data["message"] == '成功---success':
        size = data["data"]["numFound"]

    return size

In [7]:
get_size()

{'code': '0000', 'message': '成功---success', 'data': {'item': [{'admissions': '1', 'answerurl': '', 'belong': '教育部', 'central': '2', 'city_id': '1101', 'city_name': '北京市', 'code_enroll': '1000300', 'colleges_level': '', 'county_id': '110108', 'county_name': '海淀区', 'department': '1', 'doublehigh': '0', 'dual_class': '38000', 'dual_class_name': '双一流', 'f211': 1, 'f985': 1, 'hightitle': '清华大学', 'inner_rate': '65.60', 'is_recruitment': '1', 'level': '2001', 'level_name': '普通本科', 'name': '清华大学', 'nature': '36000', 'nature_name': '公办', 'outer_rate': '14.00', 'province_id': '11', 'province_name': '北京', 'rank': '', 'rank_type': '', 'rate': '96.90', 'school_id': 140, 'school_type': '6000', 'tag_name': '教育部直属', 'type': '5000', 'type_name': '综合类', 'view_month': '46716', 'view_total': '2411.5w', 'view_total_number': '24115456', 'view_week': '4631'}, {'admissions': '1', 'answerurl': '', 'belong': '教育部', 'central': '2', 'city_id': '1101', 'city_name': '北京市', 'code_enroll': '1000100', 'colleges_level'

2889

由于每次请求返回的数据是 JSON 格式，通过上述方式可以直接获取到总记录数，从而可以很方便的计算出总请求次数：  

其次， 遍历请求次数，构造每一次的请求 url，其实主要是更新下标 index  

最后则是解析每一页的数据，由于数据是 JSON 格式，可以很方便的通过 DataFrame 进行转换

In [32]:
def get_university_info(size, page_size=20):  # 2889条 /  20条/页
    page_cnt = int(size/page_size) if size%page_size==0 else int(size/page_size)+1
    print('一共{0}页数据，即将开始爬取...'.format(page_cnt))

    df_result = pd.DataFrame()
    for index in range(1, page_cnt+1):
        print('正在爬取第 {0}/{1} 页数据'.format(index, page_cnt))
        url = "https://api.zjzw.cn/web/api/?keyword=&page={0}&size=20&uri=apidata/api/gkv3/school/lists".format(index)
        res = requests.post(url, headers={'User-Agent': get_ua()})
        data = json.loads(res.text)

        if data["message"] == '成功---success':
            df_data = pd.DataFrame(data["data"]["item"])
            df_result = pd.concat([df_result, df_data])
            time.sleep(random.randint(3, 5))
    return df_result

调用主函数并保存文件即可

In [ ]:
university_df = get_university_info(2889, 20)
university_df.to_excel("../data/university_data.xlsx")

# 地址解析，反匹经纬度

地址解析主要是通过百度开发者平台，对高校的地址进行解析  

平台提供的接口是这样的：  

![](https://raw.githubusercontent.com/double-point/GraphBed/master/craw_university/02.png)  

对应的，我们将每个 **高校的地址** 作为 query 参数，将 **省份+城市** 作为 region 参数，设置 tag 为 高等院校，构造访问 url

In [34]:
def get_university_loc(url):
    """
    获取每一个站点的详细数据
    @param url:
    @return:
    """
    res = requests.get(url, headers={'User-Agent': get_ua()})
    data = json.loads(res.text)
    if len(data["results"]) > 0:
        df_data = pd.DataFrame(data["results"])
        df_data['lat'] = df_data['location'].apply(lambda x: x['lat'])
        df_data['lng'] = df_data['location'].apply(lambda x: x['lng'])
        df_data.drop('location', axis=1, inplace=True)
        # 返回数据
        bd_lng = df_data.loc[0, "lng"]
        bd_lat = df_data.loc[0, "lat"]

        return bd_lng, bd_lat

个人开发者在百度开放平台每天有 3w 次地点检索的免费配额，在使用的时候可以注意下  

本次的高校数据集大概不到 3000 条，建议在运行之前可以先写好测试 demo  


**最终的主函数如下：**  

> **需要手动添加自己的百度开发者 key**  
> .

In [35]:
if __name__ == '__main__':

    df_data = pd.read_excel("../data/university_data.xlsx")

    for row_index, data_row in df_data.iterrows():
        region = data_row['province_name']+data_row['city_name']
        query = data_row['name']
        print(query)
        url = 'http://api.map.baidu.com/place/v2/search?query={0}&tag={1}&region={2}&output=json&ak={3}'.format(
            query, "高等院校", region,  'Z7q0WBomr1GbD6HVGSD6GyBIrkqeoFhi'
        )
        try:
            # 通过百度api获取经度、纬度信息
            bd_lng, bd_lat = get_university_loc(url)
            df_data.loc[df_data['name']==query, 'bd经度'] = bd_lng
            df_data.loc[df_data['name']==query, 'bd纬度'] = bd_lat
        except:
            print("{0} 爬取失败，请注意！".format(query))
            print(url)

    # df_data.to_csv('全国高校数据（带经纬度）.csv', index=False, encoding='gbk')
    df_data.to_excel("../data/university_data_loc.xlsx")

清华大学
北京大学
浙江大学
上海交通大学
复旦大学
南京大学
中国科学技术大学
华中科技大学
武汉大学
西安交通大学
中山大学
四川大学
哈尔滨工业大学
北京航空航天大学
东南大学
北京理工大学
同济大学
中国人民大学
北京师范大学
南开大学
北京协和医学院
天津大学
山东大学
中南大学
厦门大学
西北工业大学
华南理工大学
吉林大学
电子科技大学
湖南大学
中国农业大学
华东师范大学
大连理工大学
南方科技大学
重庆大学
上海财经大学
北京科技大学
南京理工大学
南京航空航天大学
东北大学
西安电子科技大学
兰州大学
苏州大学
华中农业大学
北京交通大学
华东理工大学
上海科技大学
首都医科大学
郑州大学
中央财经大学
华中师范大学
南京医科大学
上海大学
哈尔滨工程大学
南方医科大学
暨南大学
中国政法大学
对外经济贸易大学
南京农业大学
中国石油大学（北京）
东北师范大学
武汉理工大学
西南交通大学
中国矿业大学
北京邮电大学
香港中文大学（深圳）
南京师范大学
中国地质大学（武汉）
江南大学
河海大学
中国海洋大学
北京工业大学
哈尔滨医科大学
北京化工大学
西北大学
西南财经大学
中国石油大学（华东）
陕西师范大学
天津医科大学
中南财经政法大学
深圳大学
南昌大学
浙江工业大学
中国医科大学
中国地质大学（北京）
西南大学
福州大学
西北农林科技大学
华北电力大学
中国矿业大学（北京）
东华大学
江苏大学
华南师范大学
北京外国语大学
云南大学
扬州大学
北京林业大学
中国传媒大学
北京中医药大学
合肥工业大学
宁波大学
南京工业大学
南京邮电大学
湖南师范大学
长安大学
首都师范大学
南京信息工程大学
杭州电子科技大学
湘潭大学
华南农业大学
温州医科大学
燕山大学
福建师范大学
广西大学
中国药科大学
广州大学
上海外国语大学
上海中医药大学
河南大学
大连海事大学
重庆医科大学
浙江师范大学
广东工业大学
广州医科大学
东北财经大学
上海理工大学
上海纽约大学
山西大学
外交学院
安徽大学
中央民族大学
山东师范大学
河北工业大学
青岛大学
太原理工大学
贵州大学
南京林业大学
上海师范大学
武汉科技大学
内蒙古大学
浙江理工大学
东北林业大学
福建农林大学
江西财经大学
宁波诺丁汉大学
西安理工大学
海南大学
江西师范大学
东北农业大学
河北医

### 最终数据及效果


爬取到的高校数据共 2827 条，每个数据 46 个字段  

部分数据截图如下：  

.  

![](https://raw.githubusercontent.com/double-point/GraphBed/master/craw_university/%E9%AB%98%E6%A0%A1%E6%95%B0%E6%8D%AE.png)  

.  

将高校经纬度打在地图上，分布是这样的：  

.  

![](https://raw.githubusercontent.com/double-point/GraphBed/master/craw_university/%E9%AB%98%E6%A0%A1%E5%88%86%E5%B8%83.png)  

.  

这份数据和目前教育部公布的数据还是有一些小小的出入，但是在可容忍范围之内，所以误差可以暂时忽略。  

ok，数据已经确认了，接下来就是如何分析的问题了，下节分析部分会详细介绍。  
